# Машинный перевод

**Частично автоматизированный перевод**

Программа помогает переводить части предложений, которые может, но полный перевод делает человек. Например, в специализированных областях, где есть достаточные объемы перевода и устоявшаяся терминология или шаблоны, в компании-переводчике может накопиться база уже готовых выверенных переводов и оттуда фразы можно брать и подставлять автоматически. Это упрощает перевод, передавая рутину системе, а сложные и интересные задачи решает человек.

CAT-системы (Computer-aided translation, machine-assisted translation)

[Можно еще почитать тут](https://sysblok.ru/nlp/cat-sistemy-i-budushhee-perevoda/)

**Полностью машинный перевод**

**Статистический машинный перевод**

Наиболее частый - PBMT - phrase-based machine translation. Есть пословный, синтаксический и другие. В парах предложений в большом корпусе статистически определяются соответствия кусочков предложений (фраз) и эта информация используется для перевода нового текста. Часто используется как базовая модель, если нет достаточного корпуса для нейронного.

**Rule-based**

Перевод, основанный на словарях и правилах трансформации предложений при переводе. Есть словарные соответствия и схемы, по которым перестраивается синтаксис. [Например, Apertium](https://www.apertium.org/)


**NMT - neural machine translation**

Обучается нейронная сеть на большом корпусе, которая учится предсказывать наилучший перевод.


**Как оценивается качество?**

Например, с помощью BLEU (bilingual evaluation understudy) - доля слов или чаще нграмм, которые совпали в варианте перевода и ответами (0 - все плохо, 1 - отлично, но так не бывает даже у человека). Считается, что 4-граммы наиболее хорошо описывают хорошоую человеческую речь. 


Крупные переводчики (Яндекс, Гугл) - PMBT и NMT (по возможности).


Для обучения моделей нужны параллельные корпуса с выравниванием по предложению, например, [OPUS](https://opus.nlpl.eu/)

# Выравнивание по словам

Можно научиться с помощью статистических алгоритмов определять соответствия слов (или фраз) в языке.


Будет использовать программу fast_align

In [1]:
! git clone https://github.com/clab/fast_align.git

Cloning into 'fast_align'...
remote: Enumerating objects: 213, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 213 (delta 2), reused 4 (delta 2), pack-reused 204
Receiving objects: 100% (213/213), 70.68 KiB | 5.44 MiB/s, done.
Resolving deltas: 100% (110/110), done.


Собираем ее из исходных файлов

In [6]:
! mkdir /content/fast_align/build # создаем папку для сборки
%cd fast_align/build # переходим в папку
# ! pwd
! cmake .. # собираем
! make # собираем
%cd /content/ # возвращаемся в домашнюю папку
# ! pwd


/content


Скачиваем с OPUS небольшой корпус книг

In [1]:
! wget https://object.pouta.csc.fi/OPUS-Books/v1/moses/en-ru.txt.zip

--2021-06-17 16:45:56--  https://object.pouta.csc.fi/OPUS-Books/v1/moses/en-ru.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1613419 (1.5M) [application/zip]
Saving to: ‘en-ru.txt.zip’

en-ru.txt.zip       100%[===================>]   1.54M  2.26MB/s    in 0.7s    

2021-06-17 16:45:57 (2.26 MB/s) - ‘en-ru.txt.zip’ saved [1613419/1613419]



Распаковываем архив

In [2]:
! unzip en-ru.txt.zip

Archive:  en-ru.txt.zip
  inflating: Books.en-ru.en          
  inflating: Books.en-ru.ru          
  inflating: Books.en-ru.ids         
  inflating: README                  


Преобразуем данные в тот вид, в котором они принимаются программой

In [3]:
! paste Books.en-ru.en Books.en-ru.ru -d "\t" | sed 's/\t/ ||| /' > Books.en-ru

Вот так

In [4]:
! head Books.en-ru

Anna Karenina ||| Анна Каренина
Leo Tolstoy ||| Толстой Лев Николаевич
Vengeance is mine; I will repay. ||| Мне отмщение, и аз воздам
VOLUME ONE PART I ||| ЧАСТЬ ПЕРВАЯ
CHAPTER I ||| I
ALL HAPPY FAMILIES resemble one another, but each unhappy family is unhappy in its own way. ||| Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему.
Everything was upset in the Oblonskys' house. ||| Все смешалось в доме Облонских.
The wife had discovered an intrigue between her husband and their former French governess, and declared that she would not continue to live under the same roof with him. ||| Жена узнала, что муж был в связи с бывшею в их доме француженкою-гувернанткой, и объявила мужу, что не может жить с ним в одном доме.
This state of things had now lasted for three days, and not only the husband and wife but the rest of the family and the whole household suffered from it. ||| Положение это продолжалось уже третий день и мучительно чувствовалось и самими

Копируем программу в рабочую директорию, чтобы не прописывать длинный путь

In [5]:
! mv ./fast_align ./fast_align_git # переименуем папку, а то она будет с таким же именем

mv: cannot stat './fast_align': No such file or directory


In [17]:
! cp ./fast_align_git/build/fast_align ./fast_align
! cp ./fast_align_git/build/atools ./atools

**Запускаем**

Выравнивание происходит в одну сторону, в другую и потом объединяется информация.

In [18]:
! ./fast_align -i Books.en-ru -d -o -v > forward.align


ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
.................
expected target length = source length * 0.86155
ITERATION 1
.................
  log_e likelihood: -5.8083e+06
  log_2 likelihood: -8.3796e+06
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.174998
       size counts: 1614
ITERATION 2
.................
  log_e likelihood: -2.28139e+06
  log_2 likelihood: -3.29135e+06
     cross entropy: 11.7431
        perplexity: 3427.89
      posterior p0: 0.0906316
 posterior al-feat: -0.117567
       size counts: 1614
  1  model al-feat: -0.209743 (tension=4)
  2  model al-feat: -0.164009 (tension=5.84352)
  3  model al-feat: -0.147483 (tension=6.77236)
  4  model al-feat: -0.138633 (tension=7.37068)
  5  model al-feat: -0.133147 (tension=7.79199)
  6  model al-feat: -0.129461 (tension=8.10358)
  7  model al-feat: -0.126852 (tension=8.34146)
  8  model al-feat: -0.124937 (tension=8.52716)
     final tension: 8.67456
ITERATION 3
............

In [20]:
! ./fast_align -i Books.en-ru -d -o -v -r > reverse.align

ARG=i
ARG=d
ARG=o
ARG=v
ARG=r
INITIAL PASS 
.................
expected target length = source length * 1.26205
ITERATION 1
.................
  log_e likelihood: -7.05339e+06
  log_2 likelihood: -1.01759e+07
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.172934
       size counts: 1614
ITERATION 2
.................
  log_e likelihood: -2.29682e+06
  log_2 likelihood: -3.31361e+06
     cross entropy: 9.73556
        perplexity: 852.503
      posterior p0: 0.104449
 posterior al-feat: -0.121262
       size counts: 1614
  1  model al-feat: -0.138919 (tension=4)
  2  model al-feat: -0.132396 (tension=4.35315)
  3  model al-feat: -0.128507 (tension=4.57582)
  4  model al-feat: -0.126068 (tension=4.72072)
  5  model al-feat: -0.124489 (tension=4.81684)
  6  model al-feat: -0.123445 (tension=4.88138)
  7  model al-feat: -0.122747 (tension=4.92505)
  8  model al-feat: -0.122276 (tension=4.95475)
     final tension: 4.97503
ITERATION 3
.....

In [23]:
! ./atools -i forward.align -j reverse.align -c grow-diag-final-and > result.align

Получаем такой файл. После этого мы можем соединить текст и эти индексы соответствий и достать необходимую информацию о пословных соответствиях.

In [24]:
! head result.align

0-0 1-1
0-0 0-1 1-2
0-0 1-1 2-1 2-2 3-3 4-3 5-4
0-0 1-0 2-0 2-1 3-1
0-0 1-0
0-1 1-0 2-1 2-2 3-3 4-3 5-4 7-4 7-5 8-6 9-9 11-7 11-8 14-11 15-11
0-0 1-1 2-1 3-2 4-3 5-4 6-4
1-0 2-1 2-2 3-2 4-4 4-5 5-5 6-6 7-3 8-3 9-7 10-10 11-8 11-9 13-12 14-11 14-13 15-12 16-16 17-14 18-16 19-17 20-17 21-18 22-19 23-22 24-22 25-23 26-24 27-20 28-21
1-0 3-1 4-3 5-3 6-2 7-2 8-4 9-6 10-6 10-7 13-10 14-11 15-8 15-9 16-9 18-10 19-11 20-12 21-14 22-15 23-12 23-13 23-16 24-14 27-17 28-16 29-17
0-0 1-0 2-5 3-6 4-7 5-6 6-7 8-9 9-10 10-9 11-11 12-12 13-13 15-9 16-8 17-20 18-14 21-22 22-21 24-15 24-16 25-24 26-29 29-23 30-21 31-28 33-25 34-30 34-31


**Задание**

Достать соответствия и посчитать, какие слова какими переводились.

```
{
    "семья": {
        "family": 4,
        "mother": 1
    }
}
```

## OpenNMT

Нейросетевой перевод. Это небольшой пример, который, как потом станет понятно, не очень удачный. Для нормальной работы необходим больший корпус, больше "железа" и дольше обучение. 

In [ ]:
! mkdir example

Проверяем, что кол-во строк одинаковое

In [54]:
! wc -l Books.en-ru.en

17496 Books.en-ru.en


In [55]:
! wc -l Books.en-ru.ru

17496 Books.en-ru.ru


Часть данных -  это тренировочная выборка, часть - валидационная, на которой модель проверяет себя по ходу обучения.

In [56]:
! head -n 14000 Books.en-ru.en > ./example/src-train.txt
! head -n 14000 Books.en-ru.ru > ./example/tgt-train.txt
! tail -n 3496 Books.en-ru.en > ./example/src-val.txt
! tail -n 3496 Books.en-ru.ru > ./example/tgt-val.txt

Создаем словари (какие слова учитываются), 10000 предложений для этого используются. Конфиг там же в папке на гитхабе.

In [57]:
! onmt_build_vocab -config books_en_ru.yaml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-06-17 17:17:54,551 INFO] Counter vocab from 10000 samples.
[2021-06-17 17:17:54,551 INFO] Build vocab on 10000 transformed examples/corpus.
[2021-06-17 17:17:54,563 INFO] corpus_1's transforms: TransformPipe()
[2021-06-17 17:17:54,564 INFO] Loading ParallelCorpus(example/src-train.txt, example/tgt-train.txt, align=None)...
[2021-06-17 17:17:54,882 INFO] Counters src:21023
[2021-06-17 17:17:54,882 INFO] Counters tgt:35606


**Запускаем обучение**

In [58]:
! onmt_train -config books_en_ru.yaml

[2021-06-17 17:17:58,448 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-06-17 17:17:58,448 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-06-17 17:17:58,449 INFO] Missing transforms field for valid data, set to default: [].
[2021-06-17 17:17:58,449 INFO] Parsed 2 corpora from -data.
[2021-06-17 17:17:58,449 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-06-17 17:17:58,449 INFO] Loading vocab from text file...
[2021-06-17 17:17:58,449 INFO] Loading src vocabulary from example/run/example.vocab.src
[2021-06-17 17:17:58,485 INFO] Loaded src vocab has 21023 tokens.
[2021-06-17 17:17:58,494 INFO] Loading tgt vocabulary from example/run/example.vocab.tgt
[2021-06-17 17:17:58,565 INFO] Loaded tgt vocab has 35606 tokens.
[2021-06-17 17:17:58,582 INFO] Building fields with vocab in counters...
[2021-06-17 17:17:58,651 INFO]  * tgt vocab size: 35610.
[2021-06-17 17:17:58,676 INFO]  * src vocab size:

Можем записать нужные предложения в файл и потом его передать модели.

In [72]:
! echo "He waited." > src-test.txt

In [73]:
! onmt_translate -model example/run/model_step_1000.pt -src src-test.txt -gpu 0 -verbose


[2021-06-17 17:23:05,530 INFO] Translating shard 0.
[2021-06-17 17:23:05,552 INFO] 
SENT 1: ['He', 'waited.']
PRED 1: Он видел
PRED SCORE: -7.3827

[2021-06-17 17:23:05,552 INFO] PRED AVG SCORE: -3.6913, PRED PPL: 40.0982
